In [ ]:
# To activate this environment, use                                             
#                                                                               
#     $ conda activate gee                                                      
#                                                                               
# To deactivate an active environment, use                                      
#                                                                               
#     $ conda deactivate   

: 

# Importing datasets
Import data and establish bounds for ROI

In [1]:
# Import libraries

import ee
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt




In [2]:
# Connect to GEE
ee.Authenticate()
ee.Initialize(project='wildfire-risk-ml')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [3]:
# Create Map Canvas
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Add maule region to map

regions = gpd.read_file("Regiones/Regional.shp")
# Fix CRS for regions
regions = regions.to_crs("EPSG:4326")
maule_gpd = regions[regions['Region'] == 'Región del Maule'] # Filter to Maule
#maule_gpd.to_file("maule_region.shp", driver="ESRI Shapefile")
maule = geemap.geopandas_to_ee(maule_gpd) # convert to ee object




In [ ]:
Map = geemap.Map()
Map.addLayer(maule,{},"Maule") # Add CHile to map
chile = ee.Geometry.Polygon([[
    [-75, -60],
    [-75, -17],
    [-66, -17],
    [-66, -60],
    [-75, -60]
]])
Map.centerObject(chile, 8) # Center on Chile
Map


In [5]:
regions = gpd.read_file("Regiones/Regional.shp")
regions.crs
regions = regions.to_crs("EPSG:4326")
regions.to_file("Regiones/Regional.shp")


In [6]:
# Pull in LandSat Data
#landsat_5 = ee.ImageCollection("LANDSAT/LT05/C02/T1").filterDate("1984-04-19","2011-11-08").filterBounds(maule)
#landsat_7 = ee.ImageCollection("LANDSAT/LE07/C02/T1").filterDate("2011-11-08","2013-03-18").filterBounds(maule)
landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate("2013-03-18","2018-12-31").filterBounds(maule)

# Cloud mask removal - make composite image
#composite_5 = ee.Algorithms.Landsat.simpleComposite(**
#{
#    'collection':landsat_5,
#    'asFloat': True                                     
#})
#composite_7 = ee.Algorithms.Landsat.simpleComposite(**
#{
#    'collection':landsat_7,
#    'asFloat': True                                     
#})

composite_8 = ee.Algorithms.Landsat.simpleComposite(**
{
    'collection':landsat_8,
    'asFloat': True                                     
})
#composite_7 = composite_7.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']) \
#                         .toShort()




In [7]:
#Display simple visualization

rgbVis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': .3}  # Display simple visualization
Map.addLayer(composite_8.clip(maule), rgbVis, 'RGB Viz 8')
#Map.addLayer(composite_5.clip(maule), rgbVis, '
# RGB Viz 5')
Map

Map(bottom=40188.0, center=[-36.386943716012844, -70.49999999999999], controls=(WidgetControl(options=['positi…

### Machine Learning Classification of Land Cover

In [6]:
# Define a function that scales and masks Landsat 8 surface reflectance images.
def prep_sr_l8(image):
  """Scales and masks Landsat 8 surface reflectance images."""
  # Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  qa_mask = image.select('QA_PIXEL').bitwiseAnd(0b11111).eq(0)
  saturation_mask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  def _get_factor_img(factor_names):
    factor_list = image.toDictionary().select(factor_names).values()
    return ee.Image.constant(factor_list)

  scale_img = _get_factor_img([
      'REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])
  offset_img = _get_factor_img([
      'REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
  scaled = image.select('SR_B.|ST_B10').multiply(scale_img).add(offset_img)

  # Replace original bands with scaled bands and apply masks.
  return image.addBands(scaled, None, True).updateMask(
      qa_mask).updateMask(saturation_mask)


# Make a cloud-free Landsat 8 surface reflectance composite.
l8_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2021-03-01', '2021-07-01')
    .map(prep_sr_l8)
    .median())

# Use these bands for prediction.
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# This property stores the land cover labels as consecutive
# integers starting from zero.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = l8_image.select(bands).sampleRegions(
    collection=points, properties=[label], scale=30
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = l8_image.select(bands).classify(trained)

# Display the inputs and the results.
m = geemap.Map()
m.set_center(-122.0877, 37.7880, 11)
m.add_layer(
    l8_image,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.25},
    'image',
)
m.add_layer(
    classified,
    {'min': 0, 'max': 2, 'palette': ['orange', 'green', 'blue']},
    'classification',
)
m

Map(center=[37.788, -122.0877], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…